<a href="https://colab.research.google.com/github/Akkoma/Quantum_programmation_notebook/blob/main/Bell_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install qiskit

In [ ]:
from qiskit import QuantumCircuit
from math import pi

# Initializing a qubit pair in the entangled state:
# |psi> = (|00> + |11>)/sqrt(2)
entangling_circuit = QuantumCircuit(2, 2)
entangling_circuit.h(0)
entangling_circuit.cx(0, 1)

# Adding measurements. Each measurement circuit corresponds to one
# possible arrangement of the measurement apparatus.

#ZV-measurement
zv_measurement = QuantumCircuit(2, 2)
zv_measurement.ry(pi/4, 1)
zv_measurement.measure([0, 1], [0, 1])

#ZW-measurement
zw_measurement = QuantumCircuit(2, 2)
zw_measurement.ry(-pi/4,1)
zw_measurement.measure([0, 1], [0, 1])

#XV-measurement
xv_measurement = QuantumCircuit(2, 2)
xv_measurement.h(0)
xv_measurement.ry(pi/4, 1)
xv_measurement.measure([0, 1], [0, 1])

#XW-measurement
xw_measurement = QuantumCircuit(2, 2)
xw_measurement.h(0)
xw_measurement.ry(-pi/4, 1)
xw_measurement.measure([0, 1], [0, 1])

In [ ]:
import random

# Two random sequences of 8192 binary numbers each:
# First sequence will be later used to drive Alice's measurement apparatus (Z- or X- measurement)
# Z <==> 'O', X <==> '1'
# Second sequence will be later used to drive Bob's measurement apparatus (V- or W- measurement)
# V <==> 'O', W <==> '1'
Alice_settings = [random.randint(0, 1) for i in range(8192)]
Bob_settings = [random.randint(0, 1) for i in range(8192)]

In [ ]:
from qiskit.primitives import StatevectorSampler

# Random measurements of 8192 pairs of entangled qubits:
zv_results = []
zw_results = []
xv_results = []
xw_results = []

for i in range(8192):
  if ((Alice_settings[i] == 0) and (Bob_settings[i] == 0)):
    result = StatevectorSampler().run([entangling_circuit.compose(zv_measurement)], shots = 1).result()
    data = result[0].data
    zv_results.append(list(data.c.get_counts().keys())[0])
  elif ((Alice_settings[i] == 0) and (Bob_settings[i] == 1)):
    result = StatevectorSampler().run([entangling_circuit.compose(zw_measurement)], shots = 1).result()
    data = result[0].data
    zw_results.append(list(data.c.get_counts().keys())[0])
  elif ((Alice_settings[i] == 1) and (Bob_settings[i] == 0)):
    result = StatevectorSampler().run([entangling_circuit.compose(xv_measurement)], shots = 1).result()
    data = result[0].data
    xv_results.append(list(data.c.get_counts().keys())[0])
  elif ((Alice_settings[i] == 1)  and (Bob_settings[i] == 1)):
    result = StatevectorSampler().run([entangling_circuit.compose(xw_measurement)], shots = 1).result()
    data = result[0].data
    xw_results.append(list(data.c.get_counts().keys())[0])

In [ ]:
from numpy import sum, array

# Average values
zv_results = array(zv_results)
zw_results = array(zw_results)
xv_results = array(xv_results)
xw_results = array(xw_results)

itr = 1
for config_results in [zv_results, zw_results, xv_results, xw_results]:

  probs = {}
  for output in ['00', '01', '10', '11']:
    probs[output] = sum(config_results == output) / len(config_results)


  if itr == 1:
    zv_avg = probs['00'] - probs['01'] - probs['10'] + probs['11']
    print("ZV results :")
  elif itr == 2:
    zw_avg = probs['00'] - probs['01'] - probs['10'] + probs['11']
    print("ZW results :")
  elif itr == 3:
    xv_avg = probs['00'] - probs['01'] - probs['10'] + probs['11']
    print("XV results :")
  elif itr == 4:
    xw_avg = probs['00'] - probs['01'] - probs['10'] + probs['11']
    print("XW results :")

  print('p(a = 0, b = 0) = {}'.format(probs['00']))
  print('p(a = 1, b = 0) = {}'.format(probs['01']))
  print('p(a = 0, b = 1) = {}'.format(probs['10']))
  print('p(a = 1, b = 1) = {}'.format(probs['11']))
  print('\n')

  itr += 1

print('Averages:')
print('<ZV> = {}'.format(zv_avg))
print('<ZW> = {}'.format(zw_avg))
print('<XV> = {}'.format(xv_avg))
print('<XW> = {}'.format(xw_avg))
print('\n')

S = zv_avg + zw_avg - xv_avg + xw_avg
print('S parameter:')
print('S = <ZV> + <ZW> - <XV> + <XW>')
print('S = {}'.format(S))

ZV results :
p(a = 0, b = 0) = 0.42528735632183906
p(a = 1, b = 0) = 0.07519157088122605
p(a = 0, b = 1) = 0.07614942528735633
p(a = 1, b = 1) = 0.42337164750957856


ZW results :
p(a = 0, b = 0) = 0.43267326732673267
p(a = 1, b = 0) = 0.06435643564356436
p(a = 0, b = 1) = 0.06930693069306931
p(a = 1, b = 1) = 0.43366336633663366


XV results :
p(a = 0, b = 0) = 0.0638401559454191
p(a = 1, b = 0) = 0.4264132553606238
p(a = 0, b = 1) = 0.4346978557504873
p(a = 1, b = 1) = 0.07504873294346978


XW results :
p(a = 0, b = 0) = 0.4237204724409449
p(a = 1, b = 0) = 0.06791338582677166
p(a = 0, b = 1) = 0.07086614173228346
p(a = 1, b = 1) = 0.4375


Averages:
<ZV> = 0.6973180076628352
<ZW> = 0.7326732673267327
<XV> = -0.7222222222222222
<XW> = 0.7224409448818898


S parameter:
S = <ZV> + <ZW> - <XV> + <XW>
S = 2.87465444209368
